In [1]:
#!/usr/bin/env python
from __future__ import unicode_literals
import sys
import csv
import dill as pickle
import json
import codecs
import re
import os
import pandas as pd
import import_ipynb
import TermPreprocessor2Notebook as tprep
import spacy
import ConsistencyNotebook as con

import ConsistencyDatabaseNotebook as conDB

importing Jupyter notebook from TermPreprocessor2Notebook.ipynb
importing Jupyter notebook from ConsistencyNotebook.ipynb
importing Jupyter notebook from ConsistencyDatabaseNotebook.ipynb


In [2]:
USE_DOCKER = True 
#ROOT_DIR='/ext/' if USE_DOCKER else '../../ext/'

In [3]:
def fixEntityLemma(txt, nlp):
	def getLemma(tok):
		return tok.lemma_ if tok.lemma_ != u'-PRON-' else tok.text
	#txt=txt.decode('utf-8')
	#print(txt, 'txt\n')
	doc = nlp(txt)
	#print(doc, 'doc\n')
	return ' '.join([getLemma(t) for t in doc if t.pos != spacy.symbols.DET ])

def LOG_ERROR(outputfilename, message, outputDir='../../ext/output/log_data'):
	outputDir = '../../ext/output/log_data'#os.path.join(ROOT_DIR, outputDir)
	with codecs.open(os.path.join(outputDir, outputfilename), 'a', 'utf-8') as logfile:
		logfile.write(message)
		logfile.write('\n')

def ensureUnicode(s):
	return s if type(s) == str else s


def loadFlowResults(filename, packageName, cdb, subsetNum):
	column_names = ['package_name', 'app_name', 'version_name', 'version_code', 'data_type', 'dest_domain', 'dest_ip', 'arb_number', 'privacy_policy']

	events_df = pd.read_csv(filename, header = 0, names = column_names, dtype={'app_name': str, 'privacy_policy': str})
	events_df.fillna(u'', inplace=True)

#original datamap
	#dataMap = {
		#u'aaid' : u'advertising identifier',
		#u'fingerprint' : None,
		#u'androidid' : u'android id',
		#u'geolatlon' : u'geographical location',
		#u'hwid' : u'serial number',
		#u'routerssid' : u'router ssid',
		#u'routermac' : u'mac address',
		#u'imei' : u'imei',
		#u'phone' : u'phone number',
		#u'email' : u'email address',
		#u'wifimac' : u'mac address',
		#u'invasive' : None,
		#u'package_dump' : u'application instal',
		#u'simid' : u'sim serial number',
		#u'real_name' : u'person name',
		#u'gsfid' : u'gsfid'
	#}

#sub 1 datamap:
	dataMap = {
		u'aaid' : u'advertising id',
		u'fingerprint' : None,
		u'androidid' : u'android id',
		u'hwid' : u'serial number',
		u'routerssid' : u'router ssid',
		u'routermac' : u'mac address',
		u'imei' : u'imei',
		u'wifimac' : u'mac address',
		u'invasive' : None,
		u'package_dump' : u'application instal',
		u'real_name' : u'person name',
		u'hlthwel' : u'wellness-relate datum',
		u'ftnut' : u'fitness and nutrition-relate information',
		u'gsfid' : u'gsfid',
		u'audio' : u'audio',
		u'calendar' : u'calendar',
		u'camera' : u'camera',
		u'sound' : u'sound',
		u'email' : u'email address',
		u'gallery' : u'photo',
		u'geolocation' : u'geographical location',
		u'health_wellness' : u'health and wellness',
		u'motion' : u'motion',
		u'nfc' : u'nfc',
		u'payment' : u'payment',
		u'phone' : u'phone',
		u'simid' : u'sim serial number',
		u'sms' : u'text message',
		u'socialmedia_activity' : u'social media information',
		u'video' : u'video',
		u'voice' : u'voice',
		u'music' : u'music',
		u'router' : u'router',
		u'weather' : u'weather'
	}
    
    
    
	pNameNoVers = re.sub(r'-[0-9]+$', '', packageName)

	flows = []
	for _,package_name, app_name, version_name, version_code, data_type, dest_domain, dest_ip, arb_number, privacy_policy in events_df.itertuples():
		if package_name != pNameNoVers:
			continue
		if dest_domain == '':
			dest_domain = dest_ip
		#TODO check if URL is first party or second party
		resolvedEntity = tprep.resolveUrl(dest_domain, package_name, privacy_policy)
		resolvedData = dataMap[data_type]

		if resolvedData is None:
			# log and continue
			LOG_ERROR('SkippedDataFlows_{}.log'.format(subsetNum), '{},{}'.format(data_type,packageName))
			print("skipped dataflows") # 0 count
			continue
		if resolvedEntity not in con.Entity.ontology.nodes:#TODO log
			LOG_ERROR('SkippedEntityFlows_{}.log'.format(subsetNum), '{},{}'.format(dest_domain,packageName))
			print("resolved entity not there: ", resolvedEntity) # 63 count (java.util.map - ok)          
			continue
		if resolvedData not in con.DataObject.ontology.nodes:
			LOG_ERROR('SkippedDataFlowsOnt_{}.log'.format(subsetNum), '{},{}'.format(data_type,packageName))
			print("skipped dataflowont") # 0 count
			continue

		# TODO Don't put a duplicates...
		dflow = con.DataFlow((resolvedEntity, resolvedData))
		if dflow not in flows:
			flows.append(dflow)
			cdb.insertDataFlow(resolvedEntity, resolvedData)

		cdb.insertAppDataFlow(packageName, resolvedEntity, resolvedData, dest_domain, data_type)
	print ('\tLoaded {} flows for {}'.format(len(flows), pNameNoVers))
	return flows


def shouldIgnoreSentence(s):
	mentionsChildRegex = re.compile(r'\b(child(ren)?|kids|from\sminor(s)?|under\s1[0-9]+|under\s(thirteen|fourteen|fifteen|sixteen|seventeen|eighteen)|age(s)?(\sof)?\s1[0-9]+|age(s)?(\sof)?\s(thirteen|fourteen|fifteen|sixteen|seventeen|eighteen))\b', flags=re.IGNORECASE)
	mentionsUserChoiceRegex = re.compile(r'\b(you|user)\s(.*\s)?(choose|do|decide|prefer)\s.*\s(provide|send|share|disclose)\b', flags=re.IGNORECASE)
	mentionsUserChoiceRegex2 = re.compile(r'\b((your\schoice)|(you\sdo\snot\shave\sto\sgive))\b', flags=re.IGNORECASE)
	# TODO remove false positives that discuss "except as discussed in this privacy policy / below"
#5	mentionsExceptInPrivacyPol1 = re.compile(r'\b(except\sas\s(stated|described|noted))\b', flags=re.IGNORECASE)
#6	mentionsExceptInPrivacyPol2 = re.compile(r'\b(except\sin(\sthose\slimited)?\s(cases))\b', flags=re.IGNORECASE)

#3	if mentionsChildRegex.search(s) or mentionsUserChoiceRegex.search(s) or mentionsUserChoiceRegex2.search(s) or mentionsExceptInPrivacyPol1.search(s) or mentionsExceptInPrivacyPol2.search(s):
	if mentionsChildRegex.search(s) or mentionsUserChoiceRegex.search(s) or mentionsUserChoiceRegex2.search(s):        
		return True
	return False


def loadPrivacyPolicyResults(filename, packageName, cdb, nlp, subsetNum):
	if not os.path.isfile(filename):
		return []
	policy = []
	for e,c,d,s,aLemma in pickle.load(open(filename, 'rb')):
		e = ensureUnicode(e)
		c = ensureUnicode(c)
		d = ensureUnicode(d)
		s = ensureUnicode(s)

		if c == 'not_collect' and shouldIgnoreSentence(s):
			print("should ignore sentence")
			continue

		eproc = tprep.preprocess(fixEntityLemma(e, nlp))
		if eproc in [u'user', u'you', u'person', u'consumer', u'participant']:
			print("eproc in user, you, etc.", eproc) # 10 count
			continue

		#TODO Should we try to resolve company name or ignore entity all together?
		# u'we_implicit'
		if eproc in [u'we', u'i', u'us', u'me'] or eproc in [u'app', u'mobile application', u'mobile app', u'application', u'service', u'website', u'web site', u'site'] or (e.startswith('our') and eproc in [u'app', u'mobile application', u'mobile app', u'application', u'service', u'company', u'business', u'web site', u'website', u'site']):
			eproc = u'we'

#4		if eproc == u'third_party_implicit' or eproc == u'we_implicit' or eproc == u'anyone':
#			print("e proc in third_party_implict, we_implicit, etc.")
#			continue
		if eproc == u'third_party_implicit' or eproc == u'anyone':
			eproc = u'third-party'
		if eproc == u'we_implicit':
			eproc = u'we'




#		if eproc == u'third_party_implicit':
#			eproc = u'third party'
		dproc = tprep.preprocess(d)

		ents = []
		if eproc not in con.Entity.ontology.nodes:
			res = re.sub(r'\b(and|or|and/or|\/|&)\b', u'\n', eproc)
			foundAnEnt = False
			for e in res.split('\n'):
				e = e.strip()
				if e == u'third_party_implicit' or e == u'we_implicit' or e == u'anyone':
					print("continue 1") # 0 count
					continue

				if e not in con.Entity.ontology.nodes:#This should really never happen...
					LOG_ERROR('/ext/SkippedPolicyEntities_{}.log'.format(subsetNum), e)
					print("continue 2: ", e) # 1,075 count
					continue
				ents.append(e)
		else:
			ents = [eproc]

		if len(ents) == 0:
			LOG_ERROR('/ext/SkippedPolicyEntities_{}.log'.format(subsetNum), eproc)
			print("continue 3: ", eproc) # many counts
			continue

		if dproc in con.DataObject.ontology.nodes:
			print("dproc in con.DataObject.ontology.nodes")
			if dproc in con.DataObject.ontology.nodes and dproc != con.DataObject.root:
				for e in ents:
					cdb.insertPolicy(e, c, dproc)
					cdb.insertAppPolicySentence(s, (e, c, dproc), packageName)
					policy.append((e, c, dproc, s))
		else:
			res = re.sub(r'\b(and|or|and/or|\/|&)\b', u'\n', dproc)
			for d in res.split('\n'):
				d = d.strip()
				if d not in con.DataObject.ontology.nodes or d == con.DataObject.root:#This should really never happen...
					LOG_ERROR('/ext/SkippedPolicyDataObjects_{}.log'.format(subsetNum), d)
					print("continue 4: ", d) # many counts
					continue
				for e in ents:
					if e == con.Entity.root:
						print("continue 5") # 0 counts
						continue
					cdb.insertPolicy(e, c, d)
					cdb.insertAppPolicySentence(s, (e, c, d), packageName)
					policy.append((e, c, d, s))

	return policy


def getPackageName(policyFilename):
	fname,_ = os.path.splitext(os.path.basename(policyFilename))
	return fname	

def doFilesExist(filelist):
	return all(os.path.exists(f) for f in filelist)

def main(argv):
	subsetNum = 1 #argv[1]
	#Assumes a number as input...
	consistency_database_path = '../../ext/output/db/consistency_results_{}.db'.format(subsetNum)#os.path.join(ROOT_DIR, 'output/db/consistency_results_{}.db'.format(subsetNum))
	inputDataFilename = '../../ext/datasets/{}.txt'.format(subsetNum)#os.path.join(ROOT_DIR, 'datasets/{}.txt'.format(subsetNum))
	progressFilename = '../../ext/output/log_data/{}.log'.format(subsetNum)#os.path.join(ROOT_DIR, 'output/log_data/{}.log'.format(subsetNum))

	cdb = conDB.ConsistencyDB(consistency_database_path)#'consistency_results.db'
	con.init(dataOntologyFilename=u'../../ext/data/data_ontology.pickle', entityOntologyFilename=u'../../ext/data/entity_ontology.pickle')
#	con.init_static()
	nlp = spacy.load("en_core_web_trf", exclude=["ner"])#nlp = spacy.load('/ext/NlpFinalModel')
	nlp_entity = spacy.load("../../ext/model-best")
	nlp.add_pipe("ner", source=nlp_entity)

	cdb.createTables()
	# Let's walk the policy directory now...
	progress_file = codecs.open(progressFilename, 'a', 'utf-8')
	
	files = [ line.strip() for line in codecs.open(inputDataFilename, 'r', 'utf-8') ]
	for polPath in files:
		print ('Starting', polPath)
		progress_file.write('Starting {}\n'.format(polPath))
		packageName = getPackageName(polPath)
		policy = loadPrivacyPolicyResults(os.path.join('../../ext/output/policy/', polPath), packageName, cdb, nlp, subsetNum)
		policy = [con.PolicyStatement(p) for p in set(policy) ]
		flows = loadFlowResults('../../ext/data/flows.csv', packageName, cdb, subsetNum)
		print ('\tLoaded {} policy statements for {}'.format(len(policy), packageName))


		#PolicyLint Analysis...
		policyContradictions = con.getContradictions(policy, packageName)
		for (p0, p1), contradictionIndex in policyContradictions:
			print (p0,p1,contradictionIndex, packageName)
			print (cdb.insertContradiction(contradictionIndex, packageName, p0.getTuple(), p1.getTuple()))		
		

		#PoliCheck Analysis...
		if len(flows) == 0:
			LOG_ERROR('SkippedAppsNoFlows_{}.log'.format(subsetNum), packageName)
			continue

		consistencyResults = con.checkConsistency(policy, flows)
		for cres in consistencyResults:
			flow = cres['flow']
			isConsistent,policies,contradictions = cres['consistency']

			cdb.insertConsistencyResult(flow.entity.entity, flow.data.data, packageName, isConsistent)

			numContradictions = 0
			if policies is not None:
				for i,p in enumerate(policies):
					pTuple = (p.entity.entity, p.action.action, p.data.data)
					if contradictions is not None and contradictions[i] is not None:
						for c,cnum in contradictions[i]:
							numContradictions += 1
							cTuple = (c.entity.entity, c.action.action, c.data.data)
							cdb.insertConsistencyData(flow.entity.entity, flow.data.data, packageName, pTuple, cTuple, cnum)
					else:
						cdb.insertConsistencyData(flow.entity.entity, flow.data.data, packageName, pTuple, None, -1)

			numPolicies = len(policies) if policies is not None else 0
			print ('\tFlow: {}\n\t\tIs Consistent: {}\n\t\tNum Policies: {}\n\t\tNum Contradictions: {}\n'.format(flow, isConsistent, numPolicies, numContradictions))
	
		print ('Ending', polPath)



if __name__ == '__main__':
	main(sys.argv)

Starting air.A2GIGdemoPhone.pickle
continue 4:  user of our Apps / website
dproc in con.DataObject.ontology.nodes
continue 4:  may
continue 4:  several type
dproc in con.DataObject.ontology.nodes
dproc in con.DataObject.ontology.nodes
continue 4:  user of our Apps / website
continue 4:  may
dproc in con.DataObject.ontology.nodes
dproc in con.DataObject.ontology.nodes
continue 4:  user of our Apps / website
continue 4:  several type
dproc in con.DataObject.ontology.nodes
continue 4:  may
continue 4:  may
dproc in con.DataObject.ontology.nodes
dproc in con.DataObject.ontology.nodes
continue 4:  problem
dproc in con.DataObject.ontology.nodes
continue 4:  automatic information collection technology to
continue 4:  may
continue 4:  automatic information collection technology to
continue 4:  may
dproc in con.DataObject.ontology.nodes
continue 4:  may
dproc in con.DataObject.ontology.nodes
continue 4:  to
continue 4:  may
dproc in con.DataObject.ontology.nodes
dproc in con.DataObject.ontology

True
(we, not_collect, personal) (we, collect, personal) 0 am.smarter.smarter3
we not_collect personal
we collect personal
False
(we, not_collect, personal) (we, collect, personal) 0 am.smarter.smarter3
we not_collect personal
we collect personal
False
(we, not_collect, personal) (we, collect, personal) 0 am.smarter.smarter3
we not_collect personal
we collect personal
False
(we, not_collect, personal) (we, collect, personal) 0 am.smarter.smarter3
we not_collect personal
we collect personal
False
(we, not_collect, personal) (we, collect, personal) 0 am.smarter.smarter3
we not_collect personal
we collect personal
False
(we, not_collect, personal) (we, collect, personal) 0 am.smarter.smarter3
we not_collect personal
we collect personal
False
Starting app.homey.pickle
	Loaded 0 flows for app.homey
	Loaded 0 policy statements for app.homey
Starting app.homey.pos.pickle
continue 4:  command
continue 4:  Homey
dproc in con.DataObject.ontology.nodes
continue 4:  until
continue 4:  explicit con

dproc in con.DataObject.ontology.nodes
continue 4:  judicial
continue 4:  police authority if
continue 4:  purpose of research
continue 4:  
continue 4:  judicial
continue 4:  police authority if
continue 4:  institution
dproc in con.DataObject.ontology.nodes
continue 4:  to
continue 4:  to academic
continue 4:  to
continue 4:  purchase information
continue 4:  purchase information
continue 4:  purchase information
continue 4:  institution
continue 4:  will
continue 4:  purchase information
continue 4:  to academic
continue 4:  will
continue 4:  purpose of research
continue 4:  
continue 4:  do
dproc in con.DataObject.ontology.nodes
continue 4:  do
dproc in con.DataObject.ontology.nodes
continue 4:  commercial information about our product
continue 4:  
continue 4:  permission
continue 4:  permission
we collect personal
third-party collect personal
we collect personal
third-party collect personal
	Loaded 0 flows for be.niko.nikoswitch
	Loaded 4 policy statements for be.niko.nikoswitch


	Loaded 5 flows for cn.com.broadlink.app.bestcon
	Loaded 7 policy statements for cn.com.broadlink.app.bestcon
	Flow: (we, voice)
		Is Consistent: True
		Num Policies: 4
		Num Contradictions: 0

	Flow: (we, nfc)
		Is Consistent: True
		Num Policies: 1
		Num Contradictions: 0

	Flow: (we, weather)
		Is Consistent: True
		Num Policies: 1
		Num Contradictions: 0

	Flow: (service-providers, payment)
		Is Consistent: True
		Num Policies: 1
		Num Contradictions: 0

	Flow: (service-providers, nfc)
		Is Consistent: True
		Num Policies: 1
		Num Contradictions: 0

Ending cn.com.broadlink.app.bestcon.pickle
Starting cn.com.broadlink.app.camera.pickle
continue 4:  to
dproc in con.DataObject.ontology.nodes
continue 4:  example , smart weighing scale
continue 4:  fitness tracker may
dproc in con.DataObject.ontology.nodes
continue 4:  picture
continue 4:  
continue 4:  may
continue 4:  to
dproc in con.DataObject.ontology.nodes
continue 4:  term
continue 2:  condition
continue 3:  condition
continue 2:

continue 4:  term of license agreement
dproc in con.DataObject.ontology.nodes
continue 4:  under
continue 4:  under
continue 4:  term of license agreement
continue 4:  under
dproc in con.DataObject.ontology.nodes
dproc in con.DataObject.ontology.nodes
continue 4:  under
continue 4:  term of open source license agreement
continue 4:  term of open source license agreement
continue 4:  third party
dproc in con.DataObject.ontology.nodes
continue 4:  third party
continue 4:  as be
continue 4:  as available basis
dproc in con.DataObject.ontology.nodes
continue 4:  by
continue 4:  by
continue 4:  as be
continue 4:  as available basis
continue 4:  purchase information
continue 4:  by
dproc in con.DataObject.ontology.nodes
continue 4:  such notice
continue 4:  by
dproc in con.DataObject.ontology.nodes
continue 4:  such notice
continue 4:  be
continue 4:  be
continue 4:  Awair Creative
continue 4:  warranty of kind
continue 4:  Awair Creative
continue 4:  warranty of kind
continue 4:  warranty o

...

continue 4:  service provider to
continue 4:  may
continue 4:  pay product
continue 4:  pay product
continue 4:  may
continue 4:  consent for
dproc in con.DataObject.ontology.nodes
continue 4:  third-party
third-party collect personal
we collect personal
third-party collect personal
third-party collect non-personal
we collect personal
we collect personal
we collect personal
we collect personal
third-party collect personal
we collect personal
we collect personal
we collect non-personal
third-party collect personal
we collect personal
we collect non-personal
third-party collect non-personal
third-party collect personal
we collect advertising id
third-party collect personal
we collect personal
third-party collect personal
third-party collect personal
	Loaded 0 flows for live.wallpaper3d.cyber.pipes
	Loaded 22 policy statements for live.wallpaper3d.cyber.pipes
Starting lone.wolf.wallpaper.live.keyboard.pickle
dproc in con.DataObject.ontology.nodes
continue 4:  may
continue 4:  situation
dp

dproc in con.DataObject.ontology.nodes
continue 4:  external service to
continue 4:  external service to
continue 4:  will
dproc in con.DataObject.ontology.nodes
continue 4:  will
dproc in con.DataObject.ontology.nodes
continue 4:  may
continue 4:  User provide
continue 4:  automatically collect information
continue 4:  User provide
continue 4:  automatically collect information
continue 4:  may
continue 4:  may
continue 4:  physical , electronic ,
continue 4:  procedural safeguard
continue 4:  physical , electronic ,
continue 4:  procedural safeguard
dproc in con.DataObject.ontology.nodes
dproc in con.DataObject.ontology.nodes
we collect geographical location
third-party collect non-personal
third-party collect non-personal
third-party collect non-personal
we collect non-personal
we collect non-personal
we collect non-personal
	Loaded 0 flows for net.routix.mqttdash
	Loaded 7 policy statements for net.routix.mqttdash
Starting net.superblock.pushover.pickle
continue 4:  with
dproc in c

dproc in con.DataObject.ontology.nodes
dproc in con.DataObject.ontology.nodes
continue 4:  publicly
dproc in con.DataObject.ontology.nodes
continue 4:  publicly
continue 4:  utility company
continue 4:  device partner
continue 4:  service provider
continue 4:  third party
continue 4:  may
continue 4:  utility company
continue 4:  device partner
continue 4:  service provider
continue 4:  third party
continue 4:  may
continue 4:  may
we collect personal
we collect non-personal
we collect device information
third-party collect non-personal
	Loaded 1 flows for RadioThermostat.com
	Loaded 4 policy statements for RadioThermostat.com
	Flow: (service-providers, phone)
		Is Consistent: False
		Num Policies: 0
		Num Contradictions: 0

Ending RadioThermostat.com.pickle
Starting ru.lgerp.ne.webosconnect.pickle
	Loaded 0 flows for ru.lgerp.ne.webosconnect
	Loaded 0 policy statements for ru.lgerp.ne.webosconnect
Starting se.volvo.vcc.pickle
	Loaded 1 flows for se.volvo.vcc
	Loaded 0 policy statement

Starting us.legrand.ambient.pickle
continue 4:  example
continue 4:  credit card number
continue 4:  
continue 4:  may
dproc in con.DataObject.ontology.nodes
continue 4:  may
continue 4:  example
continue 4:  may
continue 4:  device type
continue 4:  function
continue 4:  may
continue 4:  may
dproc in con.DataObject.ontology.nodes
dproc in con.DataObject.ontology.nodes
continue 4:  may
continue 4:  example
dproc in con.DataObject.ontology.nodes
continue 4:  may
dproc in con.DataObject.ontology.nodes
continue 2:  way
continue 3:  way
continue 2:  way
continue 3:  way
continue 4:  may
continue 2:  way
continue 3:  way
continue 2:  way
continue 3:  way
continue 2:  way
continue 3:  way
continue 4:  Platform
dproc in con.DataObject.ontology.nodes
continue 4:  may
continue 4:  way
continue 4:  may
continue 4:  way
dproc in con.DataObject.ontology.nodes
continue 4:  may
continue 4:  Service provider
dproc in con.DataObject.ontology.nodes
continue 4:  Service provider
continue 4:  may
dproc i

	Flow: (camera, camera)
		Is Consistent: False
		Num Policies: 0
		Num Contradictions: 0

	Flow: (audio, audio)
		Is Consistent: False
		Num Policies: 0
		Num Contradictions: 0

	Flow: (we, camera)
		Is Consistent: False
		Num Policies: 0
		Num Contradictions: 0

	Flow: (we, sound)
		Is Consistent: False
		Num Policies: 0
		Num Contradictions: 0

Ending vstc.LYLW.client.pickle
Starting wl.ehomelight.smartled.pickle
dproc in con.DataObject.ontology.nodes
continue 4:  must
we collect personal
	Loaded 1 flows for wl.ehomelight.smartled
	Loaded 1 policy statements for wl.ehomelight.smartled
	Flow: (audio, audio)
		Is Consistent: False
		Num Policies: 0
		Num Contradictions: 0

Ending wl.ehomelight.smartled.pickle
Starting wl.smartled.rgb.ahomelight.pickle
dproc in con.DataObject.ontology.nodes
continue 4:  must
we collect personal
	Loaded 1 flows for wl.smartled.rgb.ahomelight
	Loaded 1 policy statements for wl.smartled.rgb.ahomelight
	Flow: (we, audio)
		Is Consistent: True
		Num Policies

In [ ]:
with open('../../ext/output/policy/cn.com.broadlink.econtrol.international.pickle', 'rb') as f:
    viewpolicyshealth3 = pickle.load(f)
viewpolicyshealth3